# GWAS Prediction

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.decomposition import PCA
import xgboost as xgb
import plotnine as p9
import scipy.stats
from plotnine import *
from torcheval.metrics import R2Score
import torch
from scipy.stats import wilcoxon, ttest_rel, ranksums
import seaborn as sns
import matplotlib.pyplot as plt

## Data Preparation

### Load Covariates

In [2]:
def build_control_covariates(metadata):
    genesize = metadata.NPARAM.values.astype(float)
    genedensity = metadata.NPARAM.values/metadata.NSNPS.values
    inverse_mac = 1.0/metadata.MAC.values
    cov = np.stack((genesize, np.log(genesize), genedensity, np.log(genedensity), inverse_mac, np.log(inverse_mac)), axis=1)
    return cov

def munge_sigma(magma_gene_raw):
    f = open(magma_gene_raw)
    lines = list(f)[2:]
    lines = [np.asarray(line.strip('\n').split(' ')) for line in lines]
    sigmas = []
    gene_metadata = []
    gene_lists = []
    for chrom in range(1,23):
        chr_start = min(np.where([int(line[1])==chrom for line in lines])[0])
        chr_end = max(np.where([int(line[1])==chrom for line in lines])[0])
        lines_chr = lines[chr_start:chr_end+1]
        n_genes = len(lines_chr)
        sigma_chr = np.zeros([n_genes, n_genes])
        gene_NSNPs = np.zeros(n_genes)
        gene_NPARAM = np.zeros(n_genes)
        gene_MAC = np.zeros(n_genes)
        for i in range(n_genes):
            line = lines_chr[i]
            gene_NSNPs[i] = line[4]
            gene_NPARAM[i] = line[5]
            gene_MAC[i] = line[7]
            if line.shape[0] > 9:
                gene_corrs = np.asarray([float(c) for c in line[9:]])
                sigma_chr[i, i-gene_corrs.shape[0]:i] = gene_corrs
        sigma_chr = sigma_chr+sigma_chr.T+np.identity(n_genes)
        sigmas.append(sigma_chr)
        gene_metadata_chr = pd.DataFrame(data={'NSNPS': gene_NSNPs, 'NPARAM': gene_NPARAM, 'MAC': gene_MAC})
        gene_metadata.append(gene_metadata_chr)
        gene_list_chr = [line[0] for line in lines_chr]
        gene_lists.append(gene_list_chr)
    return sigmas, gene_metadata, gene_lists

In [3]:
def get_covariates(metadata, gene_lists):
    # create covariates from pops
    covariates = []
    for i in range(0, 22):
        #print(i)
        covariates.append(pd.DataFrame(build_control_covariates(metadata[i]),
                                       index = gene_lists[i],
                                       columns = ['genesize',
                                                  'log_genesize',
                                                  'genedensity',
                                                  'log_genedensity',
                                                  'inverse_mac',
                                                  'log_inverse_mac'])
                          )
    covariates = pd.concat(covariates)
    return covariates

### Load Embeddings

In [4]:
def get_embeddings(emb_name):
    emb_path = '../data/embeddings/'+emb_name+'.tsv'
    emb = pd.read_csv(emb_path, sep = "\t").set_index("gene_id")
    return emb

### Load GWAS MAGMA Scores

In [5]:
def load_magma(trait, covariates):
    magma = pd.read_csv('../data/input/GWAS/'+trait+'.genes.out', delim_whitespace=True)
    magma = magma.merge(covariates, left_on = "GENE", right_index = True)
    return magma

### Project Y to LY

In [6]:
def compute_Ls(sigmas, Y):
    Ls = []
    min_lambda=0
    for sigma in sigmas:
        W = np.linalg.eigvalsh(sigma)
        min_lambda = min(min_lambda, min(W))
    #Y = pd.read_table(args.gene_results+'.genes.out', delim_whitespace=True).ZSTAT.values
    ridge = abs(min_lambda)+.05+.9*max(0, np.var(Y)-1)
    for sigma in sigmas:
        sigma = sigma+ridge*np.identity(sigma.shape[0])
        L = np.linalg.cholesky(np.linalg.inv(sigma))
        Ls.append(L)
    return Ls

In [7]:
def project_Y(Ls, magma, gene_lists):
    LYs = []
    for i in range(22):
        L = Ls[i]
        magma_temp = magma.set_index("GENE").reindex(gene_lists[i]).reset_index()

        LYs.append(pd.DataFrame({"GENE": magma_temp.GENE, "LY": np.matmul(L, magma_temp.ZSTAT)}))
    return pd.concat(LYs)

def project_Y_back(Ls, res, gene_lists):
    LYs = []
    for i in range(22):
        L = np.linalg.inv(Ls[i])
        temp = res.set_index("GENE").reindex(gene_lists[i]).reset_index()

        LYs.append(pd.DataFrame({"GENE": temp.dropna().GENE,
                                 "pred_null": np.matmul(L[~temp.pred_LY_null.isna(), :][:, ~temp.pred_LY_null.isna()],
                                                   temp.dropna().pred_LY_null),
                                 "pred": np.matmul(L[~temp.pred_LY.isna(), :][:, ~temp.pred_LY.isna()],
                                                   temp.dropna().pred_LY),
                                 }))
    return pd.concat(LYs)

### Merge Data

In [8]:
def prepare_data(trait, emb_name):
  sigmas, metadata, gene_lists = munge_sigma('../data/input/GWAS/'+trait+'.genes.raw')
  covariates = get_covariates(metadata, gene_lists)
  
  emb = get_embeddings(emb_name)
  
  magma = load_magma(trait, covariates)
  
  # Add Ls
  Ls = compute_Ls(sigmas, magma.ZSTAT)
  magma = magma.merge(project_Y(Ls, magma, gene_lists))
  
  # Merge everything
  data = magma.merge(emb, left_on = "GENE", right_on = "gene_id")
  dict = {"sigmas": sigmas, "metadata": metadata, "gene_lists": gene_lists, "covariates": covariates, "embeddings": emb, "Ls": Ls}
  return data, dict

## Regression

Split by Chromosomes.

In [9]:
def regression_chrom_cv(dt, emb_cols, Ls, gene_lists):
    cv_results = []
    
    for chrom in range(1,23):
        reg_null = xgb.XGBRegressor(tree_method="hist", reg_lambda = 1000, reg_alpha = 100)
        reg_full = xgb.XGBRegressor(tree_method="hist", reg_lambda = 1000, reg_alpha = 100)
        
        # Null model (covariates only)
        mod_null = reg_null.fit(
            dt.query("CHR != @chrom").drop(["GENE", "CHR", "START", "STOP", "NSNPS", "NPARAM", "N", "ZSTAT", "P", "LY"], axis=1).drop(emb_cols, axis=1),
            dt.query("CHR != @chrom")['LY']
        )
        pred_null = mod_null.predict(
            dt.query("CHR == @chrom").drop(["GENE", "CHR", "START", "STOP", "NSNPS", "NPARAM", "N", "ZSTAT", "P", "LY"], axis=1).drop(emb_cols, axis=1),
        )
        
        # Full model
        mod_full = reg_full.fit(
            dt.query("CHR != @chrom").drop(["GENE", "CHR", "START", "STOP", "NSNPS", "NPARAM", "N", "ZSTAT", "P", "LY"], axis=1),
            dt.query("CHR != @chrom")['LY']
        )
        pred_full = mod_full.predict(
            dt.query("CHR == @chrom").drop(["GENE", "CHR", "START", "STOP", "NSNPS", "NPARAM", "N", "ZSTAT", "P", "LY"], axis=1),
        )
    
        results_chrom = dt.query("CHR == @chrom")[["GENE", "CHR", "START", "STOP", "NSNPS", "NPARAM", "N", "ZSTAT", "P", "LY"]]
        results_chrom['pred_LY_null'] = pred_null
        results_chrom['pred_LY'] = pred_full
    
        cv_results.append(results_chrom)
        #print(f"Chrom {chrom}: R2 (null): {scipy.stats.pearsonr(results_chrom.LY, results_chrom.pred_LY_null)[0]**2}")
        #print(f"Chrom {chrom}: R2: {scipy.stats.pearsonr(results_chrom.LY, results_chrom.pred_LY)[0]**2}")
    
    #print()
    
    cv_results = pd.concat(cv_results)
    cv_results = cv_results.merge(project_Y_back(Ls, cv_results, gene_lists))

    null_r2 = scipy.stats.pearsonr(cv_results.ZSTAT, cv_results.pred_null)[0]**2
    full_r2 = scipy.stats.pearsonr(cv_results.ZSTAT, cv_results.pred)[0]**2
    print(f"Overall R2 (null): {null_r2}")
    print(f"Overall R2: {full_r2}")

    return cv_results, null_r2, full_r2

In [10]:
def get_results(traits, model_names):
    results = []
    for trait in traits:
        print("# Trait:", trait)
        
        for model in model_names:
            print("## Model:", model)
            data, metadata = prepare_data(trait, model)
            emb_cols = metadata["embeddings"].columns.to_list()
            Ls = metadata["Ls"]
            gene_lists = metadata["gene_lists"]
            #reg_results = regression_chrom_cv(data, emb_cols, Ls, gene_lists)
            cv_results, null_r2, full_r2 = regression_chrom_cv(data, emb_cols, Ls, gene_lists)
            
            r2_null=[]
            r2_full=[]
            for i in range(1,23):
                df = cv_results.query("CHR == @i")
                r2_null.append(R2Score().update(torch.tensor(df["pred_null"].values), torch.tensor(df["ZSTAT"].values)).compute().numpy())
                r2_full.append(R2Score().update(torch.tensor(df["pred"].values), torch.tensor(df["ZSTAT"].values)).compute().numpy())
            
            res = {'trait': trait, 'name': model, 'null_r2': null_r2, 'full_r2': full_r2, 'cv_results': cv_results, 'full': np.array(r2_full), 'null': np.array(r2_null)}
            results.append(res)
            print()
    return results

## Plotting

In [11]:
def r2_per_chrom(reg_results):
    print("Per chrom R2 (full):")

    per_chrom_r2 = []
    for i in range(1, 23):
        res_tmp = reg_results.query("CHR == @i")
        r2 = scipy.stats.pearsonr(res_tmp.ZSTAT, res_tmp.pred)[0]**2
        per_chrom_r2.append(r2)
        print(r2)
    
    chromosomes = list(range(1, 23))
    df = pd.DataFrame({'Chromosome': chromosomes, 'R2': per_chrom_r2})
    
    # Plot R^2 values across chromosomes
    return (ggplot(df, aes(x='factor(Chromosome)', y='R2')) +
     geom_bar(stat='identity') +
     labs(title='R^2 Across Chromosomes', x='Chromosome', y='R2'))

In [12]:
# scatter plot (of predicted vs. true values)
def scatter(dataframe, x, y, x_label, y_label, color=None, alpha=0.1, smooth=True, facet=None):
    if color is not None:
        dataframe[color] = pd.Categorical(dataframe[color])
        p = ggplot(dataframe, aes(x, y, color=color)) + geom_point(alpha=alpha) + labs(x=x_label, y=y_label)
    else:
        p = ggplot(dataframe, aes(x, y)) + geom_point(alpha=alpha) + labs(x=x_label, y=y_label)
    if smooth:
        p += geom_smooth(method="lm")
    if facet:
        p += facet_wrap("~" + facet)
    p += geom_abline()
    return p

In [13]:
def scatterplot_model(res_df, modelname):
    model = 'Omics_d256'
    plot_data = res_df[res_df['name'] == model]
    p = ggplot(plot_data, aes('null_r2', 'full_r2', color='trait')) + geom_point() + geom_abline(intercept = 0, slope = 1) + ggtitle(model)
    return p
    
def scatterplot_all_models(res_df):
    p = ggplot(res_df, aes('null_r2', 'full_r2', color='trait')) + facet_wrap('name') + geom_point() + geom_abline(intercept = 0, slope = 1)

## Comparing Results

In [14]:
# plot r2 score
def compare_r2(dataframe=None, true = "ZSTAT", null = "pred_null", full="pred", r2_null=None, r2_full=None, r2_null_name="r2_null", r2_full_name="r2_full"):
    if dataframe is not None:
        r2_null=[]
        r2_full=[]
        for i in range(1,23):
            df = dataframe.query("CHR == @i")
            r2_null.append(R2Score().update(torch.tensor(df[null].values), torch.tensor(df[true].values)).compute().numpy())
            r2_full.append(R2Score().update(torch.tensor(df[full].values), torch.tensor(df[true].values)).compute().numpy())
    r2 = pd.DataFrame({
        r2_null_name: np.array(r2_null),
        r2_full_name: np.array(r2_full),
        "r2_delta": (np.array(r2_full) - np.array(r2_null))})
    df1 = pd.melt(r2, value_vars=[r2_null_name, r2_full_name], var_name="model", value_name="r2")
    p1 = ggplot(df1, aes("model","r2")) + geom_boxplot()
    p2 = ggplot(r2, aes("r2_delta")) + geom_histogram()
    
    w_stat, w_p_value = wilcoxon(r2[r2_null_name], r2[r2_full_name])
    #print(f'Wilcoxon p-value: {w_p_value:.4f}')
    
    # Add p-value to the plot
    p1 = p1 + labs(subtitle=f'p = {w_p_value:.4f}')
    
    return p1 #, p2

In [15]:
def compare_r2_across_models(model_results):
    data_table = pd.DataFrame()

    for model_result in model_results:
        model_name = model_result['name']
        delta_r2_column = f'delta_r2_{model_name}'
        data_table[delta_r2_column] = model_result['full'] - model_result['null']

    # Melt the DataFrame
    data_table = pd.melt(data_table, var_name='model', value_name='delta_r2')

    # Plot using plotnine
    return (ggplot(data_table, aes(x='model', y='delta_r2')) +
            geom_boxplot()) + theme(axis_text_x  = element_text(angle = 90, hjust = 1))

# Run

In [16]:
from os import walk
traits = []
filenames = next(walk("../data/input/GWAS"), (None, None, []))[2]  # [] if no file
for f in filenames:
    if (f.split(".genes.")[1] == "raw"):
        traits.append(f.split(".genes.")[0])
traits

['LDL_direct',
 'Calcium_30680',
 'HDL_cholesterol',
 'Lipoprotein_A',
 'Apolipoprotein_B',
 'Direct_bilirubin',
 'Vitamin_D',
 'HDL',
 'RBC',
 'Testosterone',
 'Alanine_aminotransferase',
 'Triglycerides',
 'Mean_corpuscular_haemoglobin',
 'Total_bilirubin',
 'IGF-1',
 'LDL_direct_adjusted_by_medication',
 'C-reactive_protein',
 'Phosphate',
 'Red_blood_cell_erythrocyte_count',
 'Albumin',
 'MCH',
 'Creatinine',
 'Cholesterol',
 'LDLC',
 'Total_protein',
 'Urate',
 'Apolipoprotein_A',
 'SHBG',
 'Glucose',
 'Calcium_100024']

In [17]:
models = ["Omics_d256", "GTEx_PCA_d64", "DepMap_PCs_d81_0.92Variance", "DepMap_ae_d128", "DepMap_DVTFM_d128", "TS_pb_PCs_d5_0.8Variance", "TS_pb_ae_d64", "TS_pb_DVTFM_d256", "rna_PCs_d4_0.95Variance", "rna_ae_d32"]
models

['Omics_d256',
 'GTEx_PCA_d64',
 'DepMap_PCs_d81_0.92Variance',
 'DepMap_ae_d128',
 'DepMap_DVTFM_d128',
 'TS_pb_PCs_d5_0.8Variance',
 'TS_pb_ae_d64',
 'TS_pb_DVTFM_d256',
 'rna_PCs_d4_0.95Variance',
 'rna_ae_d32']

In [18]:
#traits = traits[:5]
#traits = ['HDL_cholesterol', 'Testosterone', 'Vitamin_D']
results = get_results(traits, models)

# Trait: HDL_cholesterol
## Model: Omics_d256
Overall R2 (null): 0.08232776249204325
Overall R2: 0.09524026030438075

## Model: GTEx_PCA_d64
Overall R2 (null): 0.08496933277334742
Overall R2: 0.09806421815668075

## Model: DepMap_PCs_d81_0.92Variance
Overall R2 (null): 0.08280389451368463
Overall R2: 0.08502838698753164

## Model: DepMap_ae_d128
Overall R2 (null): 0.08280389451368463
Overall R2: 0.08633337134517544

## Model: DepMap_DVTFM_d128
Overall R2 (null): 0.08280389451368463
Overall R2: 0.08368785161929587

## Model: TS_pb_PCs_d5_0.8Variance
Overall R2 (null): 0.08541803349546326
Overall R2: 0.09195746495611345

## Model: TS_pb_ae_d64
Overall R2 (null): 0.08541803349546326
Overall R2: 0.09110455525719512

## Model: TS_pb_DVTFM_d256
Overall R2 (null): 0.08541803349546326
Overall R2: 0.07918513785112617

## Model: rna_PCs_d4_0.95Variance
Overall R2 (null): 0.08541803349546326
Overall R2: 0.09354428913885543

## Model: rna_ae_d32
Overall R2 (null): 0.08541803349546326
Overall R2: 0

In [19]:
res_df = pd.DataFrame.from_records(results)[['trait', 'name', 'null_r2', 'full_r2']]
res_df["delta_r2"] = res_df["full_r2"] - res_df["null_r2"]
res_df

trait                         name   null_r2   full_r2  delta_r2
0   HDL_cholesterol                   Omics_d256  0.082328  0.095240  0.012912
1   HDL_cholesterol                 GTEx_PCA_d64  0.084969  0.098064  0.013095
2   HDL_cholesterol  DepMap_PCs_d81_0.92Variance  0.082804  0.085028  0.002224
3   HDL_cholesterol               DepMap_ae_d128  0.082804  0.086333  0.003529
4   HDL_cholesterol            DepMap_DVTFM_d128  0.082804  0.083688  0.000884
5   HDL_cholesterol     TS_pb_PCs_d5_0.8Variance  0.085418  0.091957  0.006539
6   HDL_cholesterol                 TS_pb_ae_d64  0.085418  0.091105  0.005687
7   HDL_cholesterol             TS_pb_DVTFM_d256  0.085418  0.079185 -0.006233
8   HDL_cholesterol      rna_PCs_d4_0.95Variance  0.085418  0.093544  0.008126
9   HDL_cholesterol                   rna_ae_d32  0.085418  0.095351  0.009933
10     Testosterone                   Omics_d256  0.030164  0.044919  0.014755
11     Testosterone                 GTEx_PCA_d64  0.029356  0.041205  0.011849
12     Testosterone  DepMap_PCs_d81_0.92Variance  0.028354  0.031116  0.002762
13     Testosterone               DepMap_ae_d128  0.028354  0.031226  0.002872
14     Testosterone            DepMap_DVTFM_d128  0.028354  0.028730  0.000376
15     Testosterone     TS_pb_PCs_d5_0.8Variance  0.030591  0.031315  0.000724
16     Testosterone                 TS_pb_ae_d64  0.030591  0.033133  0.002542
17     Testosterone             TS_pb_DVTFM_d256  0.030591  0.026922 -0.003669
18     Testosterone      rna_PCs_d4_0.95Variance  0.030591  0.036633  0.006042
19     Testosterone                   rna_ae_d32  0.030591  0.037401  0.006810
20        Vitamin_D                   Omics_d256  0.023070  0.034506  0.011436
21        Vitamin_D                 GTEx_PCA_d64  0.023272  0.029101  0.005828
22        Vitamin_D  DepMap_PCs_d81_0.92Variance  0.025698  0.027357  0.001659
23        Vitamin_D               DepMap_ae_d128  0.025698  0.026632  0.000935
24        Vitamin_D            DepMap_DVTFM_d128  0.025698  0.030233  0.004536
25        Vitamin_D     TS_pb_PCs_d5_0.8Variance  0.025745  0.026318  0.000572
26        Vitamin_D                 TS_pb_ae_d64  0.025745  0.027407  0.001662
27        Vitamin_D             TS_pb_DVTFM_d256  0.025745  0.023148 -0.002598
28        Vitamin_D      rna_PCs_d4_0.95Variance  0.025745  0.028764  0.003019
29        Vitamin_D                   rna_ae_d32  0.025745  0.029168  0.003422

## General Comparison

In [20]:
scatterplot_all_models(res_df)

In [42]:
heatmap_data = res_df.pivot(index='trait', columns='name', values='delta_r2')
sns.set()
plt.figure(figsize=(16, 14))
h = sns.heatmap(heatmap_data, annot=True, linewidths=.5)
h.set_xticklabels(h.get_xticklabels(), rotation = 45)
plt.title('Delta RÂ²')
plt.show()

In [22]:
compare_r2_across_models(results)

<Figure Size: (640 x 480)>

In [23]:
compare_r2(r2_null_name=results[0]["name"], r2_null=results[0]["full"], r2_full_name=results[1]["name"], r2_full=results[1]["full"])

<Figure Size: (640 x 480)>

## Omics

In [24]:
scatterplot_model(res_df, "Omics_d256")

<Figure Size: (640 x 480)>

In [25]:
res_omics = results[0]['cv_results']

In [26]:
r2_per_chrom(res_omics)

Per chrom R2 (full):
0.0804065433444994
0.06725134335099976
0.18181764973308354
0.07327736017959016
0.03059141850620522
0.21989611128886694
0.05037218461677569
0.0696986472443637
0.0721257922657933
0.05950166924611103
0.10244032574834419
0.10287346419521598
0.07500605891411542
0.09388329220762026
0.0945153128694408
0.13868946663522497
0.07788948703565064
0.04909639281480897
0.03774504139548425
0.1697583535981759
0.2739760229354141
0.07864224555615076


<Figure Size: (640 x 480)>

In [27]:
compare_r2(res_omics)

<Figure Size: (640 x 480)>

## GTEx PCA

In [28]:
res_gtex_pca = results[1]['cv_results']

In [29]:
r2_per_chrom(res_gtex_pca)

Per chrom R2 (full):
0.07948893460570777
0.06065681115452023
0.1955257717627445
0.07300279851910552
0.03675052983009157
0.24809915289953316
0.054853661831332154
0.05642339138999417
0.08786142226265252
0.05449199336946204
0.1013296562029893
0.08767578742499506
0.04702820343345462
0.07505478817395267
0.10200400712668357
0.13516020288073505
0.08252471561735927
0.04653588037736781
0.03963303158023101
0.17419851988477175
0.2723440853523539
0.09369455745782088


<Figure Size: (640 x 480)>

In [30]:
compare_r2(res_gtex_pca)

<Figure Size: (640 x 480)>